In [15]:
# in this script, different targets are predicted for comparing feature engineering techniques PCA, Kernel PCA,SDAE

import numpy as np
import pandas as pd
import molml 
from molml.features import CoulombMatrix, BagOfBonds
import sys
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler 
import matplotlib.pyplot as plt
%matplotlib inline
import timeit
from sklearn.metrics import r2_score,mean_squared_error, mean_absolute_error, median_absolute_error
from sklearn.decomposition import PCA, KernelPCA, TruncatedSVD
import seaborn as sns
from IPython.core.pylabtools import figsize
from sklearn.cross_validation import StratifiedKFold
# sys.path.append("/home/peng/git/molml/")

# from molml_local.features import CoulombMatrix
# from molml_local.features import LocalCoulombMatrix
# from molml_local.kernel import AtomKernel

In [2]:
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional
from hyperopt import Trials, STATUS_OK, tpe

Using Theano backend.
Using gpu device 0: GeForce GTX 980 Ti (CNMeM is enabled with initial size: 75.0% of memory, cuDNN 5005)


In [3]:
from keras.layers import Input, Dense, Lambda, Dropout
from keras.models import Model, load_model
from keras import regularizers

In [4]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [5]:
def get_pca_features(nb_pca=16, feature_list=None):
#choose n components   
    pca_fit = PCA(n_components=nb_pca).fit(feature_list[0])
    pca_train = pca_fit.transform(feature_list[0])
    
    if np.shape(feature_list)[-1] == 3:
        pca_vali = pca_fit.transform(feature_list[1])
        pca_test = pca_fit.transform(feature_list[2])
    
        return [pca_train, pca_vali, pca_test], pca_fit
    
    elif np.shape(feature_list)[-1] == 2:
        pca_test = pca_fit.transform(feature_list[-1])   
        return [pca_train, pca_test], pca_fit
    
def get_pca_scale(pca_set):
#abs scale the pca components     
    max_abs=MaxAbsScaler()
    
    if np.shape(pca_set)[-1] == 3 :
        
        pca_set_scale=[0,0,0]
        pca_set_scale[0]= max_abs.fit_transform(pca_set[0])
        pca_set_scale[1], pca_set_scale[2]= max_abs.transform(pca_set[1]), max_abs.transform(pca_set[2])

        return pca_set_scale
    
    if np.shape(pca_set)[-1] == 2 :
        pca_set_scale=[0,0]
        pca_set_scale[0]= max_abs.fit_transform(pca_set[0])
        pca_set_scale[-1] = max_abs.transform(pca_set[-1])

        return pca_set_scale 
    
def get_nb_pca(ratio_list, thres_pca):
    
    for i in xrange(1, len(ratio_list)):
        
        if sum(ratio_list[:i])>thres_pca:
            
            nb_pca = i
            break
            
    return nb_pca      

In [16]:
def get_fit_targets(target_list = None):
    columns_list = target_list[0].columns
    train_targets = []
    vali_targets = []
    for i in columns_list:
        train_targets.append(np.array(target_list[0][i]))
        vali_targets.append(np.array(target_list[1][i]))
        
    return train_targets, vali_targets    

In [17]:
def get_attributes(element_list, coord_list):
    # produce [element, coord] lists for applying coulomb matrix and bob convertion

    fit_list = []

    for i in range(0, len(element_list)):
        fit_list.append((element_list[i], coord_list[i]))
        
    return fit_list 


def get_train_vali_test(train_list, target_list, test_szie=0.2, random_state = 32):
    
    train, test, trainlabel, testlabel = train_test_split(train_list, target_list,\
                                                      test_size=0.2, random_state = 32)

    train_, train_vali, train_label, vali_label = train_test_split(train, trainlabel, \
                                                              test_size=0.2, random_state = 32)
    
    return [train_, train_vali, test], [train_label, vali_label, testlabel]


def get_train_test(train_list, target_list, test_szie=0.2, random_state = 32):
    
    train, test, trainlabel, testlabel = train_test_split(train_list, target_list,\
                                                      test_size=0.2, random_state = 32)
    
    return [train,  test], [trainlabel, testlabel]

In [91]:
'''
def get_sdae_features(feature_list, encoding_dim = 92):
    
    noise_factor = 0.5
    
    train_noise = feature_list[0] + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=feature_list[0].shape) 
    vali_noise = feature_list[1] + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=feature_list[1].shape) 


    train_noisy = np.clip(train_noise, 0., 1.)
    vali_noisy = np.clip(vali_noise, 0., 1.)

    # this is the size of our encoded representations
      # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

    # this is our input placeholder
    
    input_dim= Input(shape=(np.shape(feature_list[0])[-1],))
    
    # "encoded" is the encoded representation of the input
    encoded = Dense(600, activation='tanh')(input_dim)
    
    encoded = Dense(encoding_dim, activation='tanh', activity_regularizer=regularizers.activity_l1(10e-5))(encoded)

    decoded = Dense(np.shape(feature_list[0])[-1], activation='linear')(encoded)


    # this model maps an input to its reconstruction
                    
    autoencoder = Model(input=input_dim, output=decoded)

    encoder = Model(input=input_dim, output=encoded)
                     
    autoencoder.compile(optimizer='sgd', loss='mse')

    start = timeit.default_timer()
                     
    history = autoencoder.fit(train_noisy, feature_list[0],
                    nb_epoch=100,
                    batch_size=100,
                    shuffle=True,
                    verbose = 0,
                    validation_data=(vali_noisy, feature_list[1])
                    )

    stop = timeit.default_timer()

    print ("The running takes %r min" %((stop-start)/60))    
    
    sdae_train, sdae_test = encoder.predict(feature_list[0]), encoder.predict(feature_list[1])
                     
    return [sdae_train, sdae_test], autoencoder, encoder, history
'''    

'\ndef get_sdae_features(feature_list, encoding_dim = 92):\n    \n    noise_factor = 0.5\n    \n    train_noise = feature_list[0] + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=feature_list[0].shape) \n    vali_noise = feature_list[1] + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=feature_list[1].shape) \n\n\n    train_noisy = np.clip(train_noise, 0., 1.)\n    vali_noisy = np.clip(vali_noise, 0., 1.)\n\n    # this is the size of our encoded representations\n      # 32 floats -> compression of factor 24.5, assuming the input is 784 floats\n\n    # this is our input placeholder\n    \n    input_dim= Input(shape=(np.shape(feature_list[0])[-1],))\n    \n    # "encoded" is the encoded representation of the input\n    encoded = Dense(600, activation=\'tanh\')(input_dim)\n    \n    encoded = Dense(encoding_dim, activation=\'tanh\', activity_regularizer=regularizers.activity_l1(10e-5))(encoded)\n\n    decoded = Dense(np.shape(feature_list[0])[-1], activation=\'linear\')

In [8]:

def data():
    
    df_data = pd.read_pickle('df_bob_target.pkl')
    feature_list = df_data['feature_list']
    
    return feature_list


def hyperas_sdae_features(feature_list, encoding_dim = 92):
    
    noise_factor = 0.5
    
    train_noise = feature_list[0] + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=feature_list[0].shape) 
    vali_noise = feature_list[1] + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=feature_list[1].shape) 


    train_noisy = np.clip(train_noise, 0., 1.)
    vali_noisy = np.clip(vali_noise, 0., 1.)

    # this is the size of our encoded representations
      # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

    # this is our input placeholder
    
    input_dim= Input(shape=(np.shape(feature_list[0])[-1],))
    
    # "encoded" is the encoded representation of the input
    
    encoded = Dense({{choice([500, 600, 700])}}, activation='tanh')(input_dim)
    
    encoded = Dense(92, activation='tanh', activity_regularizer=regularizers.activity_l1(10e-5))(encoded)

    decoded = Dense(np.shape(feature_list[0])[-1], activation='linear')(encoded)


    # this model maps an input to its reconstruction
                    
    autoencoder = Model(input=input_dim, output=decoded)

    encoder = Model(input=input_dim, output=encoded)
                     
    autoencoder.compile(optimizer='sgd', loss='mse')

    start = timeit.default_timer()
                     
    history = autoencoder.fit(train_noisy, feature_list[0],
                    nb_epoch=10,
                    batch_size=100,
                    shuffle=True,
                    verbose = 0,
                    validation_data=(vali_noisy, feature_list[1])
                    )

    stop = timeit.default_timer()

    print ("The running takes %r min" %((stop-start)/60))    
    
    sdae_train, sdae_test = encoder.predict(feature_list[0]), encoder.predict(feature_list[-1])
    
    score, mse = autoencoder.evaluate(vali_noisy, feature_list[1], verbose = 0)
    
    print ('test mse: ', mse)
                     
    return {'loss': mse, 'status':STATUS_OK, 'model':hyperas_sdae_features}

In [9]:
if __name__ == '__main__':

    best_run, best_model = optim.minimize(model=hyperas_sdae_features,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials(),
                                          notebook_name='feature_engineering_ANN'
                                         )

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import molml
except:
    pass

try:
    from molml.features import CoulombMatrix, BagOfBonds
except:
    pass

try:
    import sys
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import timeit
except:
    pass

try:
    from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, median_absolute_error
except:
    pass

try:
    from sklearn.decomposition import PCA, KernelPCA, TruncatedSVD
except:
    pass

try:
    import seaborn as sns
except:
    pass

try:
    from IPython.core.pylabtools import figsize
except:
    pass

try:
    from sklearn.cross_validation import StratifiedKFold
except:
    pass

try:
    from hyperas import optim


KeyError: -1

In [51]:
## RR use sdae to reconstruct the input and use 10-fold cv to get models
'''
sdae_set, autoencoder, encoder, history = get_sdae_features(feature_list=feature_list)

sdae_set_scale = get_pca_scale(sdae_set)

cv_model_list_sdae99, cv_abs_list_sdae99 = cross_vali_training(sdae_set_scale, target_list)

The running takes 1.757269299030304 min
Running Fold 1 / 10
Running Fold 2 / 10
Running Fold 3 / 10
Running Fold 4 / 10
Running Fold 5 / 10
Running Fold 6 / 10
Running Fold 7 / 10
Running Fold 8 / 10
Running Fold 9 / 10
Running Fold 10 / 10
The running takes 7.669215047359467 min


In [54]:
'''
dic_results_sda = {}
dic_results_sda['sda_models_92']=cv_model_list_sdae99
dic_results_sda['sda_abs_92']=cv_abs_list_sdae99
df_sda = pd.DataFrame(dic_results_sda)
df_sda.to_pickle('./Results/dic_sda.pkl')

In [16]:
#PCA1. Normalize the raw features (BOB and CM)

#feature_list, target_list = get_train_vali_test(train_list_scale_abs, df['gap'])
#feature_list, target_list = get_train_vali_test(train_list_scale_abs, df['gap'])

In [ ]:
#PCA2. get and scale the new PCA features 0.90
'''

pca_fit_ = PCA().fit(feature_list[0])

nb_pca_ = get_nb_pca(pca_fit_.explained_variance_ratio_, 0.9)

pca_set, pca_fit =get_pca_features(nb_pca=16, feature_list=feature_list)

pca_set_scale = get_pca_scale(pca_set)

cv_model_list, cv_abs_list = cross_vali_training(pca_set_scale, target_list)

In [ ]:
#PCA2. get and scale the new PCA features 0.99
'''

pca_fit_ = PCA().fit(feature_list[0])

ratio_list = pca_fit_.explained_variance_ratio_

nb_pca_ = get_nb_pca(ratio_list, 0.99)

pca_set, pca_fit =get_pca_features(nb_pca=nb_pca_, feature_list=feature_list)

pca_set_scale = get_pca_scale(pca_set)

cv_model_list_pca99, cv_abs_list_pca99 = cross_vali_training(pca_set_scale, target_list)

#dic_cv_results['pca_abs_95']= cv_abs_list_pca99

In [ ]:
#KPCA1. get and scale the new PCA features 0.99
'''

pca_fit_ = TruncatedSVD(n_components=100).fit(feature_list[1])

ratio_list = pca_fit_.explained_variance_ratio_

nb_pca_ = get_nb_pca(ratio_list, 0.9)

pca_set, pca_fit =get_pca_features(nb_pca=nb_pca_, feature_list=feature_list)

pca_set_scale = get_pca_scale(pca_set)

cv_model_list_svd, cv_abs_list_svd = cross_vali_training(pca_set_scale, target_list)

#dic_cv_results['pca_abs_95']= cv_abs_list_pca99

In [16]:
def cv_training(model, feature_train, target_train, feature_test, target_test, epochs = 100):
#PCA3. cross validation during the training  

    model.fit(feature_train, target_train,\
                                    shuffle=True,
                nb_epoch=epochs,
                batch_size=100,
                verbose = 0)
    
    metric_abs=model.evaluate(feature_test, target_test, verbose = 0) 
    
    return model, metric_abs

def cross_vali_training(training_set, target_set):
#PCA3. cross validation during the training        

    start = timeit.default_timer()

    n_folds = 10

    skf = StratifiedKFold(target_list[0], n_folds=n_folds, shuffle=True)

    cv_model_list = []
    cv_abs_list = []

    for i, (train, test) in enumerate(skf):


        print "Running Fold", i+1, "/", n_folds


        model_cv = None # Clearing the NN.

        model_cv = baseline_ann(np.shape(training_set[0])[-1])

        model_fitted, abs_cv = cv_training(model_cv,\
                                np.array(training_set[0])[train], np.array(target_set[0])[train],\
                                np.array(training_set[0])[test], np.array(target_set[0])[test], epochs=100)
        
        cv_model_list.append(model_fitted)

        cv_abs_list.append(abs_cv)
        
    stop = timeit.default_timer()

    print ("The running takes %r min" %((stop-start)/60)) 
    
    return cv_model_list, cv_abs_list    

In [ ]:
'''


pca_fit_ = TruncatedSVD(n_components=100).fit(feature_list[1])

ratio_list = pca_fit_.explained_variance_ratio_

nb_pca_ = get_nb_pca(ratio_list, 0.99)

pca_set, pca_fit =get_pca_features(nb_pca=nb_pca_, feature_list=feature_list)

pca_set_scale = get_pca_scale(pca_set)

cv_model_list_svd99, cv_abs_list_svd99 = cross_vali_training(pca_set_scale, target_list)

In [ ]:
'''

print np.mean(dic_cv_results['base_abs']), np.mean(dic_cv_results['pca_abs']),\
np.mean(dic_cv_results['pca_abs_95']), np.mean(dic_cv_results['pca_abs_99'])

In [ ]:
'''

dic_cv_results['base_models'][2].evaluate(feature_list[-1], target_list[-1])

In [ ]:
'''
dic_cv_results['base_models']=cv_model_list_base
dic_cv_results['base_abs']=cv_abs_list_base
dic_cv_results['pca_models']= cv_model_list
dic_cv_results['pca_abs']= cv_abs_list
'''

In [17]:
def baseline_ann(input_dim=666):
    
    x = Input(shape=(input_dim, ))
    h1 = Dense(1000, activation = 'tanh')(x)
    o1 = Dense(1, activation = 'linear', name='o1')(h1)
    
    model = Model(x, o1)
    
    model.compile(loss='mean_absolute_error', optimizer = 'adam')
    
    return model
    

In [ ]:
'''

# plot PCA rang
import pylab as pb
pb.axes(frameon = 0)

figsize(10,8)
plot_std = 0
color ='gray'
n=16
#print(forest.feature_importances_)
importances = pca_fit.explained_variance_ratio_[:n]
#return importances
#std=np.std([tree.feature_importances_ for tree in forest.estimators_],axis=0)
indices=np.argsort(importances)[::-1]
#print indices
print("Feature ranking:")
for f in range(12):
      print("%d. feature %d (%f)" % (f + 1, indices[f]+1, importances[indices[f]]))
#plt.figure(figsize=(8,6.5))


if plot_std == True:
    pass                                 
    #plt.bar(range(12), importances[indices],
    #color=color, yerr=std[indices], align="center")

else:
    plt.bar(range(n), importances[indices],
    color=color, align="center")

plt.xticks(range(n), indices+1, fontsize=20)
plt.yticks(fontsize = 20)
plt.xlim([-1, n])
#        plt.ylim([0.00,0.30])
plt.xlabel('Components', fontsize=24)
plt.ylabel('Eigenvalues', fontsize=24)
#ax.spines['right'].set_visible(False)
#ax.spines['top'].set_visible(False)
#plt.savefig(paper_path +"pca_rank.tif", dpi=600)
plt.show()


In [ ]:
'''
print sum(pca_fit.explained_variance_ratio_[:100])
#df_pca_train = pd.DataFrame(pca_train)
#df_pca_train.describe()

In [19]:
def test_single_dnn():

    x = Input(shape=(666, ))
    
    h1 = Dense(1000, activation = 'relu')(x)
#    h1 = Dropout(0.1)(h1)
    h1 = Dense(1000, activation = 'relu')(h1)
#    h1 = Dropout(0.1)(h1)
    h1 = Dense(1000, activation = 'relu')(h1)
    
    o1 = Dense(1, activation = 'linear', name='o1')(h1)
#     o2 = Dense(1, activation = 'linear', name='o2')(h1)
#     o3 = Dense(1, activation = 'linear', name='o3')(h1)
#     o5 = Dense(1, activation = 'linear', name='o4')(h1)
#     o5 = Dense(1, activation = 'linear', name='o5')(h1)
#     o6 = Dense(1, activation = 'linear', name='o6')(h1)
    
    model = Model(x, o1)
    
  
    model.compile(loss='mean_absolute_error', optimizer = 'adam')
    return model

In [18]:
# 0. Some setting



other_list = ['A', 'B', 'C']

elec_list = ['dipole', 'polar','spatialSize', 'homo', 'lumo', 'gap']

thermo_list = ['U0', 'U', 'H', 'G', 'Cv','zpe']

# metrics = [r2_score, mean_absolute_error, mean_squared_error, median_absolute_error]

# results_list = []

# dic_cv_results = {}

# data_path_pkl = '/home/peng/Documents/Project_C/QSAR_nlp/Dataset_qm9/'

# df = pd.read_pickle(data_path_pkl + 'filter_133k.pkl')

# fit_list = get_attributes(np.array(df['atomList']), np.array(df['atomCoords']))

# feature_methods = [CoulombMatrix(), BagOfBonds()]

# feat_co = feature_methods[1]

# train_list = feat_co.fit_transform(fit_list)

# train_list_scale = MinMaxScaler().fit_transform(train_list)

# train_list_scale_abs = MaxAbsScaler().fit_transform(train_list)

# feature_list, target_list = get_train_vali_test(train_list_scale_abs, df['gap'])

In [19]:

# 1. data importing


data_path_pkl = '/home/peng/Documents/Project_C/QSAR_nlp/Dataset_qm9/'

df = pd.read_pickle(data_path_pkl + 'filter_133k.pkl')

#print(df.columns)


In [35]:
list_all

['spatialSize',
 'polar',
 'C',
 'B',
 'A',
 'G',
 'H',
 'dipole',
 'gap',
 'U0',
 'lumo',
 'homo',
 'U',
 'zpe',
 'Cv']

In [34]:
list_all = list(set(elec_list)|set(thermo_list)|set(other_list))

In [36]:
df[list_all]

,spatialSize,polar,C,B,A,G,H,dipole,gap,U0,lumo,homo,U,zpe,Cv
0,35.3641,13.21,157.706990,157.709970,157.71180,-40.498597,-40.475117,0.0000,0.5048,-40.478930,0.1171,-0.3877,-40.476062,0.044749,6.469
1,26.1563,9.46,191.393970,293.541110,293.60975,-56.544961,-56.522082,1.6256,0.3399,-56.525887,0.0829,-0.2570,-56.523026,0.034358,6.316
2,19.0002,6.31,282.945450,437.903860,799.58812,-76.422349,-76.400922,1.8511,0.3615,-76.404702,0.0687,-0.2928,-76.401867,0.021375,6.002
3,59.5248,16.28,35.610036,35.610036,0.00000,-77.327429,-77.304583,0.0000,0.3351,-77.308427,0.0506,-0.2845,-77.305527,0.026841,8.574
4,48.7476,12.99,44.593883,44.593883,0.00000,-93.431246,-93.408425,2.8937,0.3796,-93.411888,0.0191,-0.3604,-93.409370,0.016601,6.278
5,59.9891,14.18,34.298920,38.982300,285.48839,-114.505268,-114.479802,2.1089,0.2263,-114.483613,-0.0406,-0.2670,-114.480746,0.026603,6.413
6,109.5031,23.95,19.906330,19.906490,80.46225,-79.787269,-79.759722,0.0000,0.4426,-79.764152,0.1041,-0.3385,-79.760666,0.074542,10.098
7,83.7940,16.97,23.978720,24.858720,127.83497,-115.701876,-115.674872,1.5258,0.3437,-115.679136,0.0784,-0.2653,-115.675816,0.051208,8.751
8,177.1963,28.78,8.593210,8.593230,160.28041,-116.633775,-116.604606,0.7156,0.3222,-116.609549,0.0613,-0.2609,-116.605550,0.055410,12.482
9,160.7223,24.45,9.223240,9.223270,159.03567,-132.742149,-132.713619,3.8266,0.3640,-132.718150,0.0376,-0.3264,-132.714563,0.045286,10.287


In [38]:

# 2. feature enigneering (BoB)
fit_list = get_attributes(np.array(df['atomList']), np.array(df['atomCoords']))

feature_methods = [CoulombMatrix(), BagOfBonds()]

feat_co_bob = feature_methods[1]

train_list_bob = feat_co_bob.fit_transform(fit_list)

feat_co_cm = feature_methods[0]

train_list_cm = feat_co_cm.fit_transform(fit_list)

#train_list_scale = MinMaxScaler().fit_transform(train_list)

#train_list_scale_abs = MaxAbsScaler().fit_transform(train_list)

#feature_list, target_list = get_train_vali_test(train_list_scale_abs, df['list_all'])

In [133]:
np.shape(train_list_bob[0])

(666,)

In [139]:
np.shape(feature_list_cm[0])[1]

841

In [136]:
#train_list_scale = MinMaxScaler().fit_transform(train_list_cm)

train_list_scale_abs_cm = MaxAbsScaler().fit_transform(train_list_cm)

feature_list_cm, target_list = get_train_vali_test(train_list_scale_abs_cm, df['gap'])

In [57]:
dic_features = {}

dic_features['bob_features']=train_list_bob
#dic_features['cm_features']=train_list_cm

In [50]:
dic_features['targets']= df[list_all]

In [64]:
list_bob = list(train_list_bob)
list_cm = list(train_list_cm)

In [71]:
df['bob']=list_bob

In [72]:
df['cm']= list_cm

In [73]:
df.to_pickle('df_features_targets.pkl')

In [86]:
# def train_DNN(target_single=None, model_old = None, epochs = 100):
    
#     #Run1, the single-DNN on electro_list:
#     start = timeit.default_timer()

#     #R1 generate feature and target_list:
#     feature_list, target_list = get_train_vali_test(train_list_scale, df[target_single])

#     #R3 instance model and fit
#     if model_old:
        
#         model_1 = model_old
        
#     else:
#         model_1 = test_single_dnn()

#     history = model_1.fit(feature_list[0], target_list[0],\
#                                     shuffle=True,
#                 nb_epoch=epochs,
#                 batch_size=100,
#                 verbose = 0,
#                validation_data = (feature_list[1], \
#                                    target_list[1]))

#     stop = timeit.default_timer()

#     print ("The running takes %r min" %((stop-start)/60))
    
#     return model_1, history, feature_list, target_list


# def plot_loss(history = None):
#     # plot the loss 
#     print(history.history.keys())
#     #plt.plot(history.history['o2_loss'])
#     #plt.plot(history.history['val_o2_loss'])
#     plt.plot(history.history['loss'], label = 'loss')
#     plt.plot(history.history['val_loss'], label = 'val_loss')
#     plt.legend()
#     plt.show()
    
    
# def metric_check(model_=None, feature_list=None, target_list=None):
#     # predict results and check the predicting performance 
#     pred_target_1 = np.reshape(model_.predict(feature_list[-1]),-1)
#     for metric in metrics:
#         print (str(metric), metric(pred_target_1, target_list[-1]))
       

'\ndef train_DNN(target_single=None, model_old = None, epochs = 100):\n    \n    #Run1, the single-DNN on electro_list:\n    start = timeit.default_timer()\n\n    #R1 generate feature and target_list:\n    feature_list, target_list = get_train_vali_test(train_list_scale, df[target_single])\n\n    #R3 instance model and fit\n    if model_old:\n        \n        model_1 = model_old\n        \n    else:\n        model_1 = test_single_dnn()\n\n    history = model_1.fit(feature_list[0], target_list[0],                                    shuffle=True,\n                nb_epoch=epochs,\n                batch_size=100,\n                verbose = 0,\n               validation_data = (feature_list[1],                                    target_list[1]))\n\n    stop = timeit.default_timer()\n\n    print ("The running takes %r min" %((stop-start)/60))\n    \n    return model_1, history, feature_list, target_list\n\n\ndef plot_loss(history = None):\n    # plot the loss \n    print(history.history.ke

In [84]:
'''test can be deleted'''
df_all = pd.read_pickle('df_features_targets.pkl')

In [85]:
df_only= pd.read_pickle('df_bob_target.pkl')

In [86]:
only_bob = df_only['feature_list']

In [94]:
np.shape(only_bob[0][0])

(666,)

In [95]:
allbob = np.array(df_all['bob'])

In [102]:
allbob_reshape =allbob.reshape(np.shape(allbob)[0], 666)

ValueError: total size of new array must be unchanged

In [108]:
allbob[:600].shape

(600,)

In [106]:
only_bob[0].shape

(83731, 666)

In [112]:
type(allbob[0])

numpy.ndarray

In [114]:
allbob[0]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  5.49477531,
        5.49476946,  5.49474894,  5.49474169,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [116]:
allbob_part1, allbob_part2 = allbob[0:100], allbob[100:200]

In [117]:
part12 = [allbob_part1, allbob_part2]

In [125]:
np.shape(part12[0][0])

(666,)

In [124]:
type(part12[0][0])

numpy.ndarray

In [129]:
try_1 = part12[0].reshape(100,666)

ValueError: total size of new array must be unchanged

In [126]:
len(part12[0])

100

In [140]:
df.describe()

,numAtoms,dbindex,A,B,C,dipole,polar,homo,lumo,gap,spatialSize,zpe,U0,U,H,G,Cv
count,130831.000000,130831.000000,130831.000000,130831.000000,130831.000000,130831.000000,130831.000000,130831.000000,130831.000000,130831.000000,130831.000000,130831.000000,130831.000000,130831.000000,130831.000000,130831.000000,130831.000000
mean,18.032500,66839.584976,9.966023,1.406728,1.127399,2.672953,75.281185,-0.240210,0.011835,0.252045,1189.410643,0.149090,-410.819448,-410.810977,-410.810033,-410.852852,31.620364
std,2.943715,38457.235392,1830.462974,1.600828,1.107471,1.503479,8.173831,0.021967,0.046850,0.047192,280.478157,0.033138,39.894282,39.894066,39.894066,39.894784,4.067581
min,3.000000,1.000000,0.000000,0.337120,0.331180,0.000000,6.310000,-0.428600,-0.175000,0.024600,19.000200,0.015951,-714.568061,-714.560153,-714.559209,-714.602138,6.002000
25%,16.000000,33749.500000,2.555040,1.091545,0.911495,1.577800,70.480000,-0.252600,-0.023300,0.217000,1017.431250,0.125638,-437.878804,-437.870862,-437.869918,-437.911830,28.955000
50%,18.000000,67093.000000,3.090100,1.370650,1.082030,2.475300,75.600000,-0.241100,0.012600,0.250200,1147.221100,0.148629,-416.808462,-416.800533,-416.799589,-416.841301,31.578000
75%,20.000000,100063.500000,3.836890,1.655050,1.282720,3.596350,80.610000,-0.228900,0.050900,0.289400,1309.047000,0.171397,-387.040458,-387.031217,-387.030273,-387.074521,34.298000
max,29.000000,133885.000000,619867.683140,437.903860,282.945450,29.556400,196.620000,-0.101700,0.193500,0.622100,3374.753200,0.273944,-40.478930,-40.476062,-40.475117,-40.498597,46.969000


In [141]:
df_results_sum = pd.read_pickle('./Results/dic_pca_base_svd.pkl')

Using Theano backend.
ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device 0 failed:
initCnmem: cnmemInit call failed! Reason=CNMEM_STATUS_OUT_OF_MEMORY. numdev=1

ERROR (theano.sandbox.cuda): ERROR: Not using GPU. Initialisation of device gpu failed:
initCnmem: cnmemInit call failed! Reason=CNMEM_STATUS_OUT_OF_MEMORY. numdev=1

Using Theano backend.
Using Theano backend.


AttributeError: 'module' object has no attribute 'tests'

In [ ]:
df